<a href="https://www.kaggle.com/code/shravankumar147/01-nanogpt?scriptVersionId=186236545" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-30 13:01:19--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  5.15MB/s    in 0.2s    

2024-06-30 13:01:20 (5.15 MB/s) - 'input.txt' saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [3]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [6]:
print(encode("hello"))

[46, 43, 50, 50, 53]


In [7]:
print(decode(encode("hello")))

hello


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch

print(f"No. of chars: {len(text)}")
print(f"No. of encoded integers: {len(encode(text))}") # => 1115394; 
assert len(text)==len(encode(text)) #this confirms our ecoder is working fine


No. of chars: 1115394
No. of encoded integers: 1115394


In [9]:
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype=torch.long)

print(data.shape, data.dtype)

print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# Let's now split up the data into train and validation sets
# first 90% will be train, rest val
print(f"Data Size: {len(data)}")
n = int(len(data)*0.9)

print(f"90%: {len(data[:n])}| 10%: {len(data[n:])}")

print(f"90%: {len(data[:n])} + 10%: {len(data[n:])} =  {len(data[:n]) + len(data[n:])}")
assert len(data[:n]) + len(data[n:]) == len(data)


train_data = data[:n]
val_data = data[n:]

Data Size: 1115394
90%: 1003854| 10%: 111540
90%: 1003854 + 10%: 111540 =  1115394


In [11]:
block_size = 8 # also known as context length, that a transformer see

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

print(f"input context-> target")
print("="*50)

for t in range(block_size):    
    print(f"{x[:t+1]} -> {y[t]}")

input context-> target
tensor([18]) -> 47
tensor([18, 47]) -> 56
tensor([18, 47, 56]) -> 57
tensor([18, 47, 56, 57]) -> 58
tensor([18, 47, 56, 57, 58]) -> 1
tensor([18, 47, 56, 57, 58,  1]) -> 15
tensor([18, 47, 56, 57, 58,  1, 15]) -> 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> 58


In [13]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split=='train' else val_data
    
    # This line generates batch_size random indices (ix) within the range from 0 to len(data) - block_size. 
    # The subtraction by block_size ensures there’s enough room to create sequences of length block_size.
    ix = torch.randint(low=0, high=len(data)-block_size, size=(batch_size,)) # we get batch_size random integers within the range
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x,y

In [14]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [15]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b, t]
        
        print(f"when the input is: {context.tolist()}, the target is: {target}")

when the input is: [24], the target is: 43
when the input is: [24, 43], the target is: 58
when the input is: [24, 43, 58], the target is: 5
when the input is: [24, 43, 58, 5], the target is: 57
when the input is: [24, 43, 58, 5, 57], the target is: 1
when the input is: [24, 43, 58, 5, 57, 1], the target is: 46
when the input is: [24, 43, 58, 5, 57, 1, 46], the target is: 43
when the input is: [24, 43, 58, 5, 57, 1, 46, 43], the target is: 39
when the input is: [44], the target is: 53
when the input is: [44, 53], the target is: 56
when the input is: [44, 53, 56], the target is: 1
when the input is: [44, 53, 56, 1], the target is: 58
when the input is: [44, 53, 56, 1, 58], the target is: 46
when the input is: [44, 53, 56, 1, 58, 46], the target is: 39
when the input is: [44, 53, 56, 1, 58, 46, 39], the target is: 58
when the input is: [44, 53, 56, 1, 58, 46, 39, 58], the target is: 1
when the input is: [52], the target is: 58
when the input is: [52, 58], the target is: 1
when the input i

In [16]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [18]:
emb = nn.Embedding(vocab_size, vocab_size)

Here we test the Embedding layer

It accepts a tensor as input

In [19]:
print(emb(torch.tensor(0))) # for a 1-d tensor it has generated below output
print(emb(torch.tensor(0)).shape) # the shape is 65, which are embeddings from the embedding lookup table

tensor([ 0.1808, -0.0700, -0.3596, -0.9152,  0.6258,  0.0255,  0.9545,  0.0643,
         0.3612,  1.1679, -1.3499, -0.5102,  0.2360, -0.2398, -0.9211,  1.5433,
         1.3488, -0.1396,  0.2858,  0.9651, -2.0371,  0.4931,  1.4870,  0.5910,
         0.1260, -1.5627, -1.1601, -0.3348,  0.4478, -0.8016,  1.5236,  2.5086,
        -0.6631, -0.2513,  1.0101,  0.1215,  0.1584,  1.1340, -1.1539, -0.2984,
        -0.5075, -0.9239,  0.5467, -1.4948, -1.2057,  0.5718, -0.5974, -0.6937,
         1.6455, -0.8030,  1.3514, -0.2759, -1.5108,  2.1048,  2.7630, -1.7465,
         1.4516, -1.5103,  0.8212, -0.2115,  0.7789,  1.5333,  1.6097, -0.4032,
        -0.8345], grad_fn=<EmbeddingBackward0>)
torch.Size([65])


We can pass a batch of input to the embedding layer, so let's pass our xb batch

In [20]:
xb.shape

torch.Size([4, 8])

In [21]:
logits = emb(xb)
B,T,C = logits.shape
print(B,T,C)

4 8 65


4 is the batch size

8 is the block size (time-dimention)

65 is the output embedding (channel -dimention)

Let's do some reshaping - this will come handy when we compute cross-entropy loss for our Bigram model

In [22]:
logits.view(B*T, C).shape # using view approach

torch.Size([32, 65])

In [23]:
logits.flatten(start_dim=0, end_dim=1).shape # using flatten approach

torch.Size([32, 65])

We can use any method, so I am chosing view, because I can use my pre computed B,T, and C there

In [34]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        print(logits.shape)
        if targets is None:
            loss = None
        else:
            print(targets.shape)
            
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            print(logits.shape)
            print(targets.shape)
            loss = F.cross_entropy(logits, targets)
            print(loss.shape)
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        pass
        
        

In [35]:
model = BigramLanguageModel(vocab_size)

In [36]:
model(xb)

torch.Size([4, 8, 65])


(tensor([[[ 0.4431,  0.8079,  0.1164,  ...,  0.7518,  0.3750, -1.2435],
          [-0.2245, -0.8710, -0.1792,  ..., -0.9120, -0.3722, -2.1224],
          [ 1.3565,  1.4173, -1.4363,  ..., -0.3440,  0.1705,  0.2296],
          ...,
          [-0.3801,  0.7018, -0.2113,  ...,  2.6878, -0.0383, -0.5667],
          [ 0.2069, -1.4096,  0.2898,  ...,  1.2784,  0.3223,  1.3288],
          [-0.2245, -0.8710, -0.1792,  ..., -0.9120, -0.3722, -2.1224]],
 
         [[-1.9387,  0.1355, -1.4978,  ...,  2.4457,  0.0596, -0.8411],
          [ 0.0547,  0.2618,  1.3190,  ..., -0.5641, -0.1748,  0.1328],
          [-1.0519,  0.5082,  0.5445,  ...,  1.5149,  0.7761, -2.2275],
          ...,
          [ 0.2069, -1.4096,  0.2898,  ...,  1.2784,  0.3223,  1.3288],
          [ 1.4059, -0.4747,  2.3200,  ..., -0.9030, -1.3301, -0.3194],
          [ 1.3565,  1.4173, -1.4363,  ..., -0.3440,  0.1705,  0.2296]],
 
         [[-1.9676, -0.6505, -0.3899,  ..., -0.2830,  0.7560,  0.0279],
          [ 1.3565,  1.4173,

In [39]:
logits, loss = model(xb, yb)

torch.Size([4, 8, 65])
torch.Size([4, 8])
torch.Size([32, 65])
torch.Size([32])
torch.Size([])


In [58]:
F.cross_entropy(logits, yb.view(-1))

tensor(4.5564, grad_fn=<NllLossBackward0>)

In [59]:
import numpy as np

In [62]:
-np.log(1/65)

4.174387269895637

According to the cross entropy for given vocab size = 65, the expected loss value is ~4.17, 

and our calculated cross_entropy value for given batch is ~4.55, this represents a little skew in the dataset, though it is close, so let's proceed to next

In [66]:
logits, loss = model(xb)

torch.Size([4, 8, 65])


In [74]:
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

Let's try to generate next sequence given input xb or idx

In [78]:
idx = xb 

In [79]:
logits, loss = model(idx)
# focus only on the last time step
logits = logits[:, -1, :] # becomes (B, C)
print(logits.shape)

torch.Size([4, 8, 65])
torch.Size([4, 65])


Since it is a sequence generation, so we are focusing on the last time step. That is comparable to yb, real target. 

In [80]:
# apply softmax to get probabilities
probs = F.softmax(logits, dim=-1) # (B, C)
print(probs)
print(probs.shape)

tensor([[0.0074, 0.0039, 0.0078, 0.0324, 0.0036, 0.0964, 0.0092, 0.0053, 0.0397,
         0.0156, 0.0010, 0.0069, 0.0083, 0.0530, 0.0027, 0.0578, 0.0030, 0.0091,
         0.0293, 0.0227, 0.0107, 0.0042, 0.0010, 0.0138, 0.0056, 0.0098, 0.0106,
         0.0038, 0.0115, 0.0084, 0.0140, 0.0172, 0.0229, 0.0047, 0.0196, 0.0049,
         0.0166, 0.0149, 0.0180, 0.0131, 0.0098, 0.0102, 0.0070, 0.0070, 0.0295,
         0.0340, 0.0422, 0.0158, 0.0049, 0.0121, 0.0075, 0.0105, 0.0213, 0.0045,
         0.0133, 0.0077, 0.0026, 0.0559, 0.0108, 0.0299, 0.0016, 0.0103, 0.0037,
         0.0064, 0.0011],
        [0.0432, 0.0459, 0.0026, 0.0053, 0.0087, 0.0070, 0.0148, 0.0046, 0.0090,
         0.0069, 0.0173, 0.0299, 0.0151, 0.0212, 0.0140, 0.0096, 0.0121, 0.0118,
         0.0182, 0.0034, 0.0208, 0.0037, 0.0116, 0.0041, 0.0077, 0.0073, 0.0091,
         0.0106, 0.0027, 0.0034, 0.0041, 0.0198, 0.0117, 0.0120, 0.0070, 0.0154,
         0.0074, 0.0114, 0.0242, 0.0068, 0.0204, 0.0098, 0.1098, 0.0351, 0.0094,
  

torch.multinomial returns a tensor where each row contains num_samples indices sampled from the multinomial probability distribution located in the corresponding row of tensor input.

In [81]:
# sample from the distribution
idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
print(idx_next)
print(idx_next.shape)
# append sampled index to the running sequence
idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

tensor([[46],
        [52],
        [ 5],
        [10]])
torch.Size([4, 1])


In [82]:
idx

tensor([[24, 43, 58,  5, 57,  1, 46, 43, 46],
        [44, 53, 56,  1, 58, 46, 39, 58, 52],
        [52, 58,  1, 58, 46, 39, 58,  1,  5],
        [25, 17, 27, 10,  0, 21,  1, 54, 10]])

In [83]:
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

In [90]:
torch.cat([idx_next, yb[:,-1].view(-1,1)], dim=1)

tensor([[46, 39],
        [52,  1],
        [ 5, 46],
        [10, 39]])

The first row(idx_next) represents the output generated by the model for given idx or(xb) as input. 

The last row (yb) represents the target or expected output. 